Вам необходимо написать функцию, которая будет основана на поиске по сайту habr.com. Функция в качестве параметра должна принимать список запросов для поиска (например, ['python', 'анализ данных']) и на основе материалов, попавших в результаты поиска по каждому запросу, возвращать датафрейм вида:  

<дата> - <заголовок> - <ссылка на материал>  

В рамках задания предполагается работа только с одной (первой) страницей результатов поисковой выдачи для каждого запроса. Материалы в датафрейме не должны дублироваться, если они попадали в результаты поиска для нескольких запросов из списка.  

# 1 Импорт библиотек

In [77]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [51]:
url = 'https://habr.com/ru/search/'
params = {
    'q': ['python', 'анализ данных']
}
res = requests.get(url, params)
soup = BeautifulSoup(res.text)
soup

<!DOCTYPE html>
<html lang="ru">
<head>
<title>Результаты поиска по запросу «python,анализ данных» / Хабр</title>
<link data-hid="2a79c45" href="/img/habr_ru.png" rel="image_src"/>
<link data-hid="e3fa780" href="https://habr.com/ru/search/" rel="canonical"/>
<meta content="/img/habr_ru.png" itemprop="image"/>
<meta content="/img/habr_ru.png" property="og:image"/>
<meta content="1200" property="og:image:width"/>
<meta content="630" property="og:image:height"/>
<meta content="/img/habr_ru.png" property="aiturec:image"/>
<meta content="/img/habr_ru.png" name="twitter:image"/>
<meta content="/img/habr_ru.png?format=vk" property="vk:image"/>
<meta content="444736788986613" property="fb:app_id"/>
<meta content="472597926099084" property="fb:pages"/>
<meta content="summary_large_image" name="twitter:card"/>
<meta content="@habr_com" name="twitter:site"/>
<meta content="ru_RU" property="og:locale"/>
<meta content="website" property="og:type"/>
<meta content="Хабр" property="og:site_name"/>
<me

# 2 Прописываем параметры запроса

In [76]:
url = 'https://habr.com/ru/search/'

def search_habr(queries):
    all_results = []

    for query in queries:
        print(f"Поиск: {query}")
        params = {'q': query}

# 3 Выполняем запрос и парсинг

In [75]:
        response = requests.get(url, params=params)
        soup = BeautifulSoup(response.text, 'html.parser')

        # Находим все статьи
        articles = soup.find_all('article', class_='tm-articles-list__item')

        # Обрабатываем все статьи в одном цикле
        for article in articles:
            # Ищем заголовок и ссылку
            title_link = article.find('a', class_='tm-title__link')
            if title_link:
                title = title_link.text.strip()
                link = 'https://habr.com' + title_link['href']

                # Ищем дату
                time_elem = article.find('time')
                date = time_elem.text.strip() if time_elem else ""

                # Добавляем результат
                all_results.append({
                    'дата': date,
                    'заголовок': title,
                    'ссылка на материал': link
                })

        time.sleep(1)

NameError: name 'all_results' is not defined

# 4 Созданем датафрейм и удаленяем дубликаты

In [ ]:
# Создаем датафрейм с русскими названиями колонок
    if all_results:
        df = pd.DataFrame(all_results)
        df = df.drop_duplicates(subset=['ссылка на материал'])
        return df
    else:
        return pd.DataFrame(columns=['дата', 'заголовок', 'ссылка на материал'])

# 5 Пример использования функции

In [ ]:
# Список запросов для поиска
search_queries = ['python', 'анализ данных']

# Выполняем поиск
result_df = search_habr(search_queries)

# Показываем таблицу с русскими заголовками
print(f"Найдено статей: {len(result_df)}")
print("\nТаблица результатов:")
print(result_df.to_string(index=False))

# 6 Целая функция, которая будет собирать все ссылки на новости по запросу c главной страницы сайта в виде таблицы

In [73]:
url = 'https://habr.com/ru/search/'

def search_habr(queries):
    all_results = []

    for query in queries:
        print(f"Поиск: {query}")
        params = {'q': query}

        response = requests.get(url, params=params)
        soup = BeautifulSoup(response.text, 'html.parser')

        articles = soup.find_all('article', class_='tm-articles-list__item')

        for article in articles:
            title_link = article.find('a', class_='tm-title__link')
            if title_link:
                title = title_link.text.strip()
                link = 'https://habr.com' + title_link['href']

                time_elem = article.find('time')
                date = time_elem.text.strip() if time_elem else ""

                all_results.append({
                    'дата': date,
                    'заголовок': title,
                    'ссылка на материал': link
                })

        time.sleep(1)

    if all_results:
        df = pd.DataFrame(all_results)
        df = df.drop_duplicates(subset=['ссылка на материал'])
        return df
    else:
        return pd.DataFrame(columns=['дата', 'заголовок', 'ссылка на материал'])

search_queries = ['python', 'анализ данных']
result_df = search_habr(search_queries)

print(f"Найдено статей: {len(result_df)}")
print("\nТаблица результатов:")
print(result_df.to_string(index=False))

Поиск: python
Поиск: анализ данных
Найдено статей: 40

Таблица результатов:
                 дата                                                                                                               заголовок                                           ссылка на материал
     8  окт   в 04:02                                                                                                       Релиз Python 3.14                             https://habr.com/ru/news/954394/
20  янв  2022 в 15:37                                                                   Курс «Python для инженеров». Старт 3 потока 31 января             https://habr.com/ru/companies/slurm/news/646825/
24  дек  2024 в 06:20                 Один из ключевых разработчиков CPython Тим Питерс вернулся в сообщество Python после трехмесячного бана                             https://habr.com/ru/news/869196/
13  дек  2021 в 06:00                                          Жаждущим автоматизации: открытый урок «ChatOps c 